Zelle 1 – Markdown
# 🧩 Code Overview – Python Data Processor

## Ziel dieses Notebooks
Dieses Notebook erklärt den Aufbau des Pakets `pydata_processor`:

- Welche Module es gibt (`processor`, `cli`, `__init__`)
- Welche Funktionen wichtig sind
- Wie man die Python-API direkt nutzt
- Wie das CLI intern darauf zugreift

Es ergänzt die bereits vorhandenen Notebooks:
- `01_overview.ipynb` (Projektüberblick)
- `03_usage.ipynb` (Nutzung / Beispiele)

Zelle 2 – Markdown: Paketstruktur
## 1. Paketstruktur

Das Python-Paket liegt im Ordner `src/pydata_processor`:

```text
pydata_processor/
├── __init__.py      # Paket-Metadaten, öffentliche API
├── processor.py     # Kernlogik für Lesen / Transformieren / Schreiben
└── cli.py           # Command Line Interface (Typer)


__init__.py definiert __version__ und __all__ = ["processor"]

processor.py enthält reine Datenlogik

cli.py ruft die Funktionen aus processor auf und bietet ein CLI

In [ ]:
---

### 🟢 Zelle 3 – Code: Paket importieren

```python
import pydata_processor
from pydata_processor import processor

print("Package version:", pydata_processor.__version__)

🟢 Zelle 4 – Markdown: Übersicht processor.py
## 2. Modul `processor.py` – Kernfunktionen

Das Modul `processor` enthält reine Datenverarbeitungslogik.

### Wichtige Funktionen

| Funktion            | Zweck                                               |
|---------------------|-----------------------------------------------------|
| `_read(path)`       | CSV/JSON einlesen                                  |
| `_write(df, path)`  | DataFrame als CSV/JSON speichern                   |
| `clean_columns(df)` | Spaltennamen zu `snake_case` normalisieren         |
| `drop_null_rows(df)`| Vollständig leere Zeilen entfernen                 |
| `dedupe(df, subset)`| Duplikate entfernen                                |
| `filter_query(df,q)`| DataFrame mit pandas-Query filtern                 |
| `summary(df)`       | Dict mit `rows`, `columns`, `nulls`, `dtypes`      |
| `load_transform(...)` | Einlesen + Säubern + Filtern (gibt DataFrame)   |
| `process(...)`      | Voller ETL: einlesen, säubern, filtern, speichern, Rückgabe `summary` |

Die Funktionen sind so aufgebaut, dass sie auch ohne CLI direkt aus Python
verwendet werden können.

In [ ]:
🟢 Zelle 5 – Code: kleines API-Beispiel
from pydata_processor import processor

df = processor.load_transform("data/data.csv")
summary = processor.summary(df)

df.head(), summary


(Das zeigt dir direkt die ersten Zeilen + das Summary-Dict.)

🟢 Zelle 6 – Markdown: cli.py Überblick
## 3. Modul `cli.py` – Command Line Interface

Das Modul `cli` nutzt `typer`, um drei Befehle bereitzustellen:

- `summary` – gibt ein JSON-Summary zum Datensatz aus
- `convert` – konvertiert CSV ↔ JSON ohne Filter
- `filter` – säubert, dedupliziert, filtert und speichert

Beispielaufrufe (aus dem Projekt-Hauptordner):

```bash
python -m pydata_processor.cli summary data/data.csv

python -m pydata_processor.cli convert data/data.csv output.json

python -m pydata_processor.cli filter data/data.csv clean_output.csv \
    --query "age > 25" --subset "id,name"


Intern ruft das CLI lediglich Funktionen aus processor.py auf
(load_transform und process) und formatiert die Ausgaben für das Terminal.

### 🟢 Zelle 7 – Markdown: Zusammenspiel

```markdown
## 4. Zusammenspiel von API und CLI

- Die **Python-API** (Funktionen in `processor.py`) ist unabhängig vom CLI.
- Das **CLI** (`cli.py`) ist eine dünne Schicht oben drauf:
  - parst Argumente (Typer)
  - ruft `processor.process()` oder `processor.load_transform()` auf
  - gibt Ergebnisse als Text/JSON aus

Dadurch kann das Projekt:
- sowohl als **Bibliothek** in anderen Projekten verwendet werden
- als auch als **Kommandozeilen-Tool** genutzt werden